<a href="https://www.kaggle.com/code/khushboos/iwildcamera-research-competition?scriptVersionId=118335981" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, json, codecs
import cv2 as cv
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, Dense, MaxPool2D, Activation, Dropout, Flatten, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
np.random.seed(921)
from keras.utils import np_utils
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ----------------Training meta data--------------------------------
with codecs.open("/kaggle/input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_train_annotations.json",'r', encoding='utf-8', errors = 'ignore') as f :
    train_anot_meta = json.load(f)
    
#--------------- Testing meta data --------------------------------   
with codecs.open("/kaggle/input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_test_information.json",'r', encoding='utf-8', errors='ignore') as f:
    test_anot_meta = json.load(f)
    
#---------------- Detection meta data--------------------------------
with codecs.open("/kaggle/input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_mdv4_detections.json",'r', encoding='utf-8', errors='ignore') as f:
    detection_data = json.load(f)
    
#--------------- GPS location meta data-------------------------------------
with codecs.open("/kaggle/input/iwildcam2022-fgvc9/metadata/metadata/gps_locations.json",'r', encoding='utf-8', errors='ignore') as f :
    gps_location = json.load(f)
# ("/kaggle/input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_train_annotations.json")

In [ ]:
print(f"Training anotations : {train_anot_meta.keys()}\
        \nTesting Anotation : {test_anot_meta.keys()}\
        \nGps Location : gps_location.keys()\
        \ndetection : {detection_data.keys()}")

In [ ]:
# -----------------Training meta data in pandas dataframe--------------------------------  
train_images = pd.DataFrame(train_anot_meta['images'])
train_anotations = pd.DataFrame(train_anot_meta['annotations'])
train_categories = pd.DataFrame(train_anot_meta['categories'])
print(f"Train images Shape : {train_images.shape}\
        \nAnnotation Shape : {train_anotations.shape}\
        \nTrain_Categories : {train_categories.shape}")

In [ ]:
# -----------------Test meta data in the Pandas datarame-------------------------------------
test_images = pd.DataFrame(test_anot_meta['images'])
print(f"Test_images shape :{test_images.shape}")

In [ ]:
train_images

In [ ]:
train_anotations

In [ ]:
train_categories

In [ ]:
# Let's check the number of training samles
print(f"Training Sample Length :{len(os.listdir('/kaggle/input/iwildcam2022-fgvc9/train/train'))}\
        \nTesting Sample Length :{len(os.listdir('/kaggle/input/iwildcam2022-fgvc9/test/test'))}")
# so the Number of samples are same for both training samples and for the metadata i.e. 201399

# Images of an animals 

In [ ]:
# reading the first image from the training sample and showing it down.
img = cv.imread("/kaggle/input/iwildcam2022-fgvc9/train/train/86760c00-21bc-11ea-a13a-137349068a90.jpg")
print(f"Images Shape : {img.shape}")
plt.figure(figsize=(10,10))
plt.imshow(img)

In [ ]:
train_anotations['image_id']

# Resize Images

In [ ]:
#-------Taking only 20000 data from the training set-----------------
images = train_anotations.image_id[:20000]
labels = train_anotations.category_id[:20000]

#--- Reshaping the images ------------------------------
train_images = []
for i in images:
    rd_img = cv.imread("/kaggle/input/iwildcam2022-fgvc9/train/train/"+ i +".jpg")
    res_img = cv.resize(rd_img, (32,32), interpolation=cv.INTER_AREA)
    train_images.append(res_img)
    
# Convert it into the numpy array
train_images = np.array(train_images)

In [ ]:
# train_images

# Model Creation

In [ ]:
#-------------------- Model Creation --------------------------------------------------------
model = Sequential()

#---- 1st Convolution Layer---------
model.add(Convolution2D(filters=32, kernel_size = (3,3), input_shape=(32, 32,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate = 0.35))

#----- 1st Pooling layer-------------
model.add(MaxPool2D(pool_size = (2,2), padding='same'))

#----- 2nd Convolution Layer----------
model.add(Convolution2D(filters= 64, kernel_size=(3,3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate=0.45))

#------2nd pooling Layer--------------
model.add(MaxPool2D(pool_size=(2,2), padding='same'))
model.add(Flatten())

#---- 1st Dense Layer-----------------
model.add(Dense(1024, activation = 'relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(rate=0.75))

#------2nd Dense Layer---------------
model.add(Dense(max(labels)+1, activation='softmax'))

#---------------Model Compilation------------------------------------------
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# Training Data Splitting

In [ ]:
#---------Train test split ---------------------------------
X_train, X_test, Y_train, y_test = train_test_split(train_images, labels, test_size = 0.2)

In [ ]:
#----------- Changeg datatype and reshape the data----------------
Y_train=Y_train.astype(int)
y_test=y_test.astype(int)

#---------- Reshape Y------------------
Y_train=np.array(Y_train).reshape(-1,1)
y_test=np.array(y_test).reshape(-1,1)

# -------- Reshape X ------------------
X_train=X_train.reshape(-1,32,32,3)/255
X_test=X_test.reshape(-1,32,32,3)/255

#-----------Convert it into Categorical data -------------------
Y_train=np_utils.to_categorical(Y_train,num_classes=max(labels)+1)
y_test=np_utils.to_categorical(y_test,num_classes=max(labels)+1)

# Training

In [ ]:
#------------ Train the data--------------------
train_history = model.fit(X_train,Y_train,validation_split=0.2,epochs=30,batch_size=200,verbose=1)

#----------- Calculate Accuracy --------------------
accuracy = model.evaluate(X_test,y_test,verbose=1)
print("test accuracy:",accuracy[1])

In [ ]:
#------------   Plot the Training accuracy and validation Accuracy -------------------
def show_train_history(train_history,train_accuracy,val_accuracy):
        plt.plot(train_history.history[train_accuracy])
        plt.plot(train_history.history[val_accuracy])
        plt.title('Train History')
        plt.ylabel('train')
        plt.xlabel('Epoch')
        plt.legend(['train','validation'],loc='upper left')
        plt.show()

show_train_history(train_history,'accuracy','val_accuracy') #acc:accuracy for training set. val_acc:accuracy for validation.

# Prediction

In [ ]:
#----------------- Test Predict (as this test images were from the training data) ----------------
prediction  = model.predict(X_test)
print(prediction)

In [ ]:
32*32*3

In [ ]:
print(f"{prediction.shape}, {prediction[0]}")

In [ ]:
test_images

In [ ]:
#--------------------- test the original test data------------------------
images = test_images.id[:20000]
# test_img_id =test_images.id[:30000]

test_image = []
for i in images:
    img = cv.imread("/kaggle/input/iwildcam2022-fgvc9/test/test/"+ i +".jpg")
    res_img = cv.resize(img, (32,32),interpolation=cv.INTER_AREA)
    test_image.append(res_img)
    
# Convert it into the numpy array
test_image = np.array(test_image)

In [ ]:
# -------- Predicting the original Test data------------
test_prediction = model.predict(test_image)

In [ ]:
test_prediction

In [ ]:
test_prediction.shape

In [ ]:
data = {"lists": list(images)}
submit = pd.DataFrame({'Id':images,'Predicted': list(test_prediction)})
submit.to_csv('submission.csv',index=False)

In [ ]:
pd.read_csv("/kaggle/working/submission.csv")